# Finding Climate Modes with EOFs

---

## Overview
In this notebook, we will identify and plot a few different modes of climate variability with the help of an EOF package that interfaces with Xarray called [`xeofs`](https://github.com/nicrie/xeofs).

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [Intro to EOFs](eof-intro) | Helpful | |

- **Time to learn**: 30 minutes

---

## Imports

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.colors import CenteredNorm
from scipy import stats, signal
from cartopy import crs as ccrs, feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from xeofs.xarray import EOF

## Preparing the data

We will use the monthly [Extended Reconstructed Sea Surface Temperature](https://www.ncei.noaa.gov/products/extended-reconstructed-sst) (ERSST) version 3b dataset from NOAA, which is included locally in this cookbook.

In [ ]:
sst = xr.open_dataset('./data/sst.mnmean.nc')
sst

Define a helpful function to take global averages:

In [ ]:
def global_average(data):
    weights = np.cos(np.deg2rad(data.lat))
    data_weighted = data.weighted(weights)
    return data_weighted.mean(dim=['lat', 'lon'], skipna=True)

Here we compute the SST anomaly by subtracting out the average of each month using Xarray's `.groupby()` method. This deseasonalizes the data, which is important because climate modes will get buried under the magnitude of the seasonal changes in SST.

In [ ]:
sst_clim = sst.groupby('time.month')
ssta = sst_clim - sst_clim.mean(dim='time')

To see what happens when we skip this step, let's do an EOF analysis on the unmodified monthly data:

In [ ]:
s_model = EOF(sst.sst, n_modes=4, dim=['time'], weights='coslat')
s_model.solve()
s_eofs = s_model.eofs()
s_pcs = s_model.pcs()
s_expvar = s_model.explained_variance_ratio()

In [ ]:
s_eofs.plot(col='mode')

In [ ]:
s_pcs.plot(col='mode')

In [ ]:
s_expvar

For EOF1, which explains about 86% of the variance, we can see there is interhemispheric asymmetry, and PC1 has a period of one year. This is the seasonal cycle. EOF3 and EOF4 are clearly picking up the global warming signal with some other variability mixed in, and EOF2 seems to be some combination of both. Here we have identified something else we will want to remove from the data if we are currently interested in looking at internal variability&mdash;the long-term warming. We can detrend the data by removing the global average SST anomaly:

In [ ]:
ssta_dt = (ssta - global_average(ssta)).squeeze()

Let's find the global EOFs again but with the deseasonalized, detrended data:

In [ ]:
ds_model = EOF(ssta_dt.sst, n_modes=4, dim=['time'], weights='coslat')
ds_model.solve()
ds_eofs = ds_model.eofs()
ds_pcs = ds_model.pcs()
ds_expvar = ds_model.explained_variance_ratio()

In [ ]:
ds_eofs.plot(col='mode')

In [ ]:
ds_pcs.plot(col='mode')

In [ ]:
ds_expvar

Now we can see some modes of variability! EOF1 looks like ENSO or IPO, and EOF2 looks like it is picking up a pattern of recent warming where the Southern Ocean and southeastern Pacific are slightly cooling. EOF3 shows PDO in the North Pacific. There is a lot going on in each of these maps, so to get a clearer index of some of these modes, we can restrict our domain. 

## El Niño Southern Oscillation (ENSO)

Here we restrict our domain to the equatorial Pacific. You can [read more about ENSO here](https://www.ncei.noaa.gov/access/monitoring/enso/).

In [ ]:
ep_ssta_dt = ssta_dt.where((ssta_dt.lat < 30) & (ssta_dt.lat > -30) & (ssta_dt.lon > 120) & (ssta_dt.lon < 290), drop=True)

In [ ]:
ep_model = EOF(ep_ssta_dt.sst, n_modes=4, dim=['time'], norm=True, weights='coslat')
ep_model.solve()
ep_eofs = ep_model.eofs()
ep_pcs = ep_model.pcs()
ep_expvar = ep_model.explained_variance_ratio()

In [ ]:
ep_eofs.plot(col='mode')

In [ ]:
ep_pcs.plot(col='mode')

In [ ]:
ep_expvar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 2), dpi=130)
plt.fill_between(ep_pcs.time, ep_pcs.isel(mode=0).where(ep_pcs.isel(mode=0) > 0), color='r')
plt.fill_between(ep_pcs.time, ep_pcs.isel(mode=0).where(ep_pcs.isel(mode=0) < 0), color='b')
plt.ylabel('PC')
plt.xlabel('Time')
plt.xlim(ep_pcs.time.min(), ep_pcs.time.max())
plt.grid(linestyle=':')
plt.title('ENSO Index (detrended equatorial Pacific SSTA EOF1)')

## Pacific Decadal Oscillation (PDO)

In [ ]:
np_ssta_dt = ssta_dt.where((ssta_dt.lat < 70) & (ssta_dt.lat > 20) & (ssta_dt.lon > 120) & (ssta_dt.lon < 260), drop=True)

In [ ]:
np_model = EOF(np_ssta_dt.sst, n_modes=4, dim=['time'], norm=True, weights='coslat')
np_model.solve()
np_eofs = np_model.eofs()
np_pcs = np_model.pcs()
np_expvar = np_model.explained_variance_ratio()

In [ ]:
np_eofs.plot(col='mode')

In [ ]:
np_pcs.plot(col='mode')

In [ ]:
np_expvar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 2), dpi=130)
plt.fill_between(np_pcs.time, -np_pcs.isel(mode=0).where(np_pcs.isel(mode=0) < 0), color='r')
plt.fill_between(np_pcs.time, -np_pcs.isel(mode=0).where(np_pcs.isel(mode=0) > 0), color='b')
plt.plot(np_pcs.time, -np_pcs.isel(mode=0).rolling(time=48, center=True).mean(), color='k', linewidth=2)
plt.ylabel('PC')
plt.xlabel('Time')
plt.xlim(np_pcs.time.min(), np_pcs.time.max())
plt.grid(linestyle=':')
plt.title('PDO Index (detrended North Pacific SSTA EOF1)')

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!